<a href="https://colab.research.google.com/github/SiavashMomeni/RTS_Project/blob/main/RTS_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import time
from collections import defaultdict
import heapq
import random
import math
from scipy.optimize import linprog
import itertools
import torch
import os
from concurrent.futures import ProcessPoolExecutor
import multiprocessing as mp
from tqdm import tqdm
from google.colab import drive

# تنظیمات عمومی
plt.rcParams['font.family'] = 'Tahoma'
plt.rcParams['axes.unicode_minus'] = False
def plot_all_results(results_df, analysis_df, output_dir='results'):
    """ترسیم تمام نمودارهای مورد نیاز و ذخیره در مسیر مشخص"""
    # ایجاد دایرکتوری نتایج اگر وجود نداشته باشد
    os.makedirs(output_dir, exist_ok=True)

    # 1. نمودار تغییرات زمان پاسخ
    plt.figure(figsize=(14, 8))
    for policy in results_df['policy'].unique():
        subset = results_df[results_df['policy'] == policy]
        plt.plot(subset['num_tasks'], subset['avg_response'], 'o-', label=policy)

    plt.xlabel('Number of Tasks')
    plt.ylabel('Average Response Time (ms)')
    plt.title('Impact of Task Count on Response Time')
    plt.legend()
    plt.grid(True)
    plt.savefig(f'{output_dir}/response_time_vs_tasks.png', dpi=300)
    plt.close()

    # 2. نمودار نرخ رعایت مهلت
    plt.figure(figsize=(14, 8))
    cores = results_df['cores'].unique()
    width = 0.2
    x = np.arange(len(cores))

    for i, policy in enumerate(results_df['policy'].unique()):
        success_rates = []
        for core in cores:
            subset = results_df[(results_df['policy'] == policy) & (results_df['cores'] == core)]
            if not subset.empty:
                success_rates.append(subset['success_rate'].mean() * 100)
            else:
                success_rates.append(0)
        plt.bar(x + i*width, success_rates, width=width, label=policy)

    plt.xticks(x + width, cores)
    plt.xlabel('Number of Cores')
    plt.ylabel('Deadline Meeting Rate (%)')
    plt.title('Deadline Meeting Rate in Different Systems')
    plt.legend()
    plt.grid(True, axis='y')
    plt.savefig(f'{output_dir}/deadline_success_rate.png', dpi=300)
    plt.close()

    # 3. تأثیر بهره‌وری بر زمان پاسخ و انرژی
    fig, ax1 = plt.subplots(figsize=(14, 8))

    for policy in results_df['policy'].unique():
        subset = results_df[(results_df['policy'] == policy) & (results_df['cores'] == 8)]
        ax1.plot(subset['utilization'], subset['avg_response'], 'o-', label=f'{policy} (Response)')

    ax2 = ax1.twinx()
    for policy in results_df['policy'].unique():
        subset = results_df[(results_df['policy'] == policy) & (results_df['cores'] == 8)]
        ax2.plot(subset['utilization'], subset['energy'], 's--', label=f'{policy} (Energy)')

    ax1.set_xlabel('CPU Utilization')
    ax1.set_ylabel('Average Response Time (ms)')
    ax2.set_ylabel('Energy Consumption (J)')
    plt.title('Impact of Utilization on Response Time and Energy')
    fig.legend(loc='upper center', bbox_to_anchor=(0.5, 0.95), ncol=3)
    plt.grid(True)
    plt.savefig(f'{output_dir}/utilization_impact.png', dpi=300, bbox_inches='tight')
    plt.close()

    # 4. تحلیل تأخیرهای زمانبندی
    plt.figure(figsize=(14, 8))
    for policy in results_df['policy'].unique():
        subset = results_df[results_df['policy'] == policy]
        plt.plot(subset['num_tasks'], subset['avg_waiting'], 'D-', label=policy)

    plt.xlabel('Number of Tasks')
    plt.ylabel('Average Waiting Time (ms)')
    plt.title('Scheduling-Induced Delays')
    plt.legend()
    plt.grid(True)
    plt.savefig(f'{output_dir}/scheduling_delays.png', dpi=300)
    plt.close()

    # 5. سربار پردازشی
    plt.figure(figsize=(14, 8))
    for policy in results_df['policy'].unique():
        subset = results_df[results_df['policy'] == policy]
        plt.plot(subset['num_tasks'], subset['overhead'], 's--', label=policy)

    plt.xlabel('Number of Tasks')
    plt.ylabel('Scheduling Overhead (%)')
    plt.title('Processing Overhead of Different Methods')
    plt.legend()
    plt.grid(True)
    plt.savefig(f'{output_dir}/processing_overhead.png', dpi=300)
    plt.close()

    # 6. نتایج تحلیل زمان پاسخ
    plt.figure(figsize=(14, 8))
    plt.plot(analysis_df['cores'], analysis_df['closed_form'], 'o-', label='Closed-Form')
    plt.plot(analysis_df['cores'], analysis_df['milp'], 's--', label='MILP')
    plt.plot(analysis_df['cores'], analysis_df['simulation'], 'D-.', label='Simulation')

    plt.xlabel('Number of Cores')
    plt.ylabel('Average Response Time (ms)')
    plt.title('Response Time Analysis Methods Comparison')
    plt.legend()
    plt.grid(True)
    plt.savefig(f'{output_dir}/analysis_methods_comparison.png', dpi=300)
    plt.close()

# تنظیمات عمومی
plt.rcParams['font.family'] = 'Tahoma'
plt.rcParams['axes.unicode_minus'] = False

# ==================== کلاس‌های اصلی (بهینه‌سازی شده برای GPU) ====================
class Bundle:
    """کلاس بسته وظیفه با نیاز هسته‌ای متغیر"""
    def __init__(self, num_cores, wcet):
        self.num_cores = num_cores
        self.wcet = wcet
        self.remaining = wcet
        self.start_time = -1

class GangTask:
    """کلاس وظیفه گروهی (Gang Task)"""
    def __init__(self, task_id, period, wcet_total, num_bundles, criticality=1):
        self.id = task_id
        self.period = period
        self.deadline = period
        self.release_time = 0
        self.bundles = []
        self.bundle_index = 0
        self.finish_time = None
        self.response_time = None
        self.missed_deadline = False
        self.assigned_cores = []

        # تولید بسته‌ها با نیاز هسته‌ای متغیر
        bundle_wcets = np.random.dirichlet(np.ones(num_bundles)) * wcet_total
        for i in range(num_bundles):
            cores = random.randint(1, 8)
            self.bundles.append(Bundle(cores, bundle_wcets[i]))

    def __lt__(self, other):
        return self.id < other.id

    def current_bundle(self):
        if self.bundle_index < len(self.bundles):
            return self.bundles[self.bundle_index]
        return None

    def advance_bundle(self):
        self.bundle_index += 1
        return self.current_bundle() is not None

    def total_workload(self):
        return sum(b.wcet for b in self.bundles)

    def max_cores_required(self):
        return max(b.num_cores for b in self.bundles)

class Core:
    """کلاس هسته پردازشی"""
    def __init__(self, core_id, speed=1.0):
        self.id = core_id
        self.speed = speed
        self.task = None
        self.idle = True
        self.utilization = 0
        self.energy_consumed = 0

    def execute(self, duration):
        if self.task:
            self.utilization += duration
            self.energy_consumed += duration * 0.2

    def reset(self):
        self.task = None
        self.idle = True
        self.utilization = 0
        self.energy_consumed = 0

class GangScheduler:
    """شبیه‌ساز پیشرفته زمان‌بندی Gang Tasks با بهینه‌سازی GPU"""
    def __init__(self, num_cores, scheduling_policy='FIFO', allocation_policy='FFD', migration_policy='None'):
        self.num_cores = num_cores
        self.scheduling_policy = scheduling_policy
        self.allocation_policy = allocation_policy
        self.migration_policy = migration_policy
        self.cores = [Core(i) for i in range(num_cores)]
        self.ready_queue = []
        self.running_tasks = []
        self.finished_tasks = []
        self.time = 0
        self.event_queue = []
        self.metrics = {
            'response_times': [],
            'deadline_misses': 0,
            'energy_consumption': 0.0,
            'core_utilization': [0] * num_cores,
            'scheduling_overhead': 0.0,
            'waiting_times': [],
            'migration_count': 0
        }
        self.core_assignments = defaultdict(list)
        self.scheduling_start_time = 0

        # استفاده از GPU برای محاسبات سنگین
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        print(f"Using device: {self.device}")

    def add_task(self, task):
        heapq.heappush(self.event_queue, (task.release_time, task.id, 'release', task))

    def release_task(self, task):
        task.release_time = self.time
        heapq.heappush(self.event_queue, (self.time + task.period, task.id, 'release', task))
        self.ready_queue.append(task)

    def schedule(self):
        start_time = time.time()

        if self.scheduling_policy == 'FIFO':
            self.ready_queue.sort(key=lambda t: t.release_time)
        elif self.scheduling_policy == 'LDF':
            self.ready_queue.sort(key=lambda t: t.deadline, reverse=True)
        elif self.scheduling_policy == 'HEFT':
            # محاسبه رتبه HEFT روی GPU
            self.calculate_heft_ranks_gpu()
            self.ready_queue.sort(key=lambda t: t.rank, reverse=True)

        for task in self.ready_queue[:]:
            bundle = task.current_bundle()
            if not bundle:
                continue

            allocated = self.allocate_cores(task, bundle)
            if allocated:
                self.ready_queue.remove(task)
                self.running_tasks.append(task)

        self.metrics['scheduling_overhead'] += time.time() - start_time

    def calculate_heft_ranks_gpu(self):
        """محاسبه رتبه HEFT با استفاده از GPU"""
        if not self.ready_queue:
            return

        # آماده سازی داده‌ها برای GPU
        periods = torch.tensor([task.period for task in self.ready_queue], dtype=torch.float32, device=self.device)
        wcet_sums = torch.tensor([sum(b.wcet for b in task.bundles) for task in self.ready_queue],
                                dtype=torch.float32, device=self.device)
        num_bundles = torch.tensor([len(task.bundles) for task in self.ready_queue],
                                  dtype=torch.float32, device=self.device)
        avg_cores = torch.tensor([sum(b.num_cores for b in task.bundles)/len(task.bundles)
                                 for task in self.ready_queue], dtype=torch.float32, device=self.device)

        # محاسبه رتبه‌ها روی GPU
        avg_execution = wcet_sums / num_bundles
        ranks = periods / (avg_execution * avg_cores)

        # انتقال نتایج به CPU و ذخیره در اشیاء وظیفه
        ranks_cpu = ranks.cpu().numpy()
        for i, task in enumerate(self.ready_queue):
            task.rank = ranks_cpu[i]

    def allocate_cores(self, task, bundle):
        required = bundle.num_cores
        available_cores = [c for c in self.cores if c.idle]

        if len(available_cores) < required:
            return False

        # سیاست‌های تخصیص
        if self.allocation_policy == 'FFD':
            allocated_cores = available_cores[:required]
        elif self.allocation_policy == 'BFD':
            allocated_cores = sorted(available_cores, key=lambda c: c.speed)[:required]
        elif self.allocation_policy == 'WFD':
            allocated_cores = sorted(available_cores, key=lambda c: c.speed, reverse=True)[:required]
        else:
            allocated_cores = available_cores[:required]

        # اختصاص وظیفه به هسته‌ها
        task.assigned_cores = allocated_cores
        bundle.start_time = self.time

        for core in allocated_cores:
            core.task = task
            core.idle = False

        finish_time = self.time + bundle.remaining
        heapq.heappush(self.event_queue, (finish_time, task.id, 'finish', task))
        return True

    def finish_bundle(self, task):
        bundle = task.current_bundle()
        if not bundle:
            return

        if bundle.start_time < 0:
            bundle.start_time = self.time

        actual_duration = self.time - bundle.start_time

        for core in task.assigned_cores:
            core.execute(actual_duration)
            core.task = None
            core.idle = True
            self.metrics['core_utilization'][core.id] += actual_duration

        # محاسبه انرژی مصرفی
        active_power = 1.0
        idle_power = 0.1
        self.metrics['energy_consumption'] += actual_duration * active_power * bundle.num_cores
        self.metrics['energy_consumption'] += actual_duration * idle_power * (self.num_cores - bundle.num_cores)

        if task.advance_bundle():
            self.ready_queue.append(task)
            task.assigned_cores = []
        else:
            task.finish_time = self.time
            response = task.finish_time - task.release_time
            self.metrics['response_times'].append(response)
            task.response_time = response

            if task.finish_time > task.deadline:
                self.metrics['deadline_misses'] += 1
                task.missed_deadline = True

            self.running_tasks.remove(task)
            self.finished_tasks.append(task)

    def run(self, sim_time):
        self.metrics['scheduling_overhead'] = 0.0
        self.scheduling_start_time = time.time()

        while self.time < sim_time and self.event_queue:
            time_event, _, event_type, task = heapq.heappop(self.event_queue)
            self.time = time_event

            if event_type == 'release':
                self.release_task(task)
            elif event_type == 'finish':
                self.finish_bundle(task)

            self.schedule()

        self.calculate_final_metrics()

    def calculate_final_metrics(self):
        if self.metrics['response_times']:
            self.metrics['avg_response'] = np.mean(self.metrics['response_times'])
            self.metrics['max_response'] = np.max(self.metrics['response_times'])
        else:
            self.metrics['avg_response'] = 0
            self.metrics['max_response'] = 0

        total_tasks = len(self.finished_tasks)
        if total_tasks > 0:
            self.metrics['miss_rate'] = self.metrics['deadline_misses'] / total_tasks
            self.metrics['success_rate'] = 1 - self.metrics['miss_rate']

            if self.metrics['waiting_times']:
                self.metrics['avg_waiting'] = np.mean(self.metrics['waiting_times'])
            else:
                self.metrics['avg_waiting'] = 0
        else:
            self.metrics['miss_rate'] = 0
            self.metrics['success_rate'] = 0
            self.metrics['avg_waiting'] = 0

        total_sim_time = time.time() - self.scheduling_start_time
        if total_sim_time > 0:
            self.metrics['overhead_percentage'] = (self.metrics['scheduling_overhead'] / total_sim_time) * 100
        else:
            self.metrics['overhead_percentage'] = 0

# ==================== توابع شبیه‌سازی و تجزیه و تحلیل ====================
def generate_tasks(num_tasks, utilization, num_bundles_range=(1, 5)):
    tasks = []
    total_utilization = 0
    periods = [10, 20, 50, 100, 200, 500, 1000]

    for i in range(num_tasks):
        period = random.choice(periods)
        num_bundles = random.randint(*num_bundles_range)

        task_util = utilization / num_tasks * random.uniform(0.8, 1.2)
        wcet_total = task_util * period

        task = GangTask(i, period, wcet_total, num_bundles)
        tasks.append(task)
        total_utilization += task_util

    actual_util = total_utilization
    for task in tasks:
        task.bundles[0].wcet *= utilization / actual_util

    return tasks

def run_single_experiment(config):
    """اجرای یک آزمایش مستقل"""
    tasks = generate_tasks(config['num_tasks'], config.get('utilization', 0.7))
    scheduler = GangScheduler(
        config['cores'],
        scheduling_policy=config['policy'],
        allocation_policy=config.get('allocation', 'FFD')
    )

    for task in tasks:
        scheduler.add_task(task)

    scheduler.run(config.get('sim_time', 1000))

    return {
        'config': config,
        'metrics': scheduler.metrics
    }

def run_comprehensive_experiments():
    """اجرای آزمایش‌های جامع فاز دوم با موازی‌سازی"""
    from concurrent.futures import ProcessPoolExecutor
    from tqdm import tqdm
    import multiprocessing as mp

    # کاهش محدوده آزمایش‌ها برای اجرای سریع‌تر
    core_counts = [4, 8, 16, 32]
    task_counts = [10, 50, 100, 200, 400, 600]
    utilizations = [0.25, 0.5, 0.75]
    policies = ['FIFO', 'LDF', 'HEFT']
    allocation_policies = ['FFD', 'BFD', 'WFD', 'RR']

    # تولید پیکربندی‌های آزمایش
    configs = []

    # آزمایش 1: تأثیر تعداد وظایف
    for n in task_counts:
        for cores in core_counts:
            for policy in policies:
                for alloc in allocation_policies:
                    configs.append({
                        'num_tasks': n,
                        'cores': cores,
                        'policy': policy,
                        'allocation': alloc,
                        'sim_time': 7000  # کاهش زمان شبیه‌سازی
                    })

    # آزمایش 2: تأثیر بهره‌وری
    for util in utilizations:
        for cores in core_counts:
            for policy in policies:
                configs.append({
                    'num_tasks': 100,
                    'cores': cores,
                    'policy': policy,
                    'utilization': util,
                    'sim_time': 2000  # کاهش زمان شبیه‌سازی
                })

    # اجرای موازی آزمایش‌ها
    results = []
    cpu_count = mp.cpu_count()
    print(f"Starting parallel execution with {cpu_count} processes...")

    with ProcessPoolExecutor(max_workers=cpu_count) as executor:
        futures = [executor.submit(run_single_experiment, config) for config in configs]

        for future in tqdm(futures, total=len(configs), desc="Running experiments"):
            results.append(future.result())

    # تبدیل نتایج به DataFrame
    data = []
    for res in results:
        config = res['config']
        metrics = res['metrics']
        data.append({
            'num_tasks': config.get('num_tasks', 0),
            'cores': config.get('cores', 0),
            'policy': config.get('policy', ''),
            'allocation': config.get('allocation', ''),
            'utilization': config.get('utilization', 0.7),
            'avg_response': metrics.get('avg_response', 0),
            'success_rate': metrics.get('success_rate', 0),
            'energy': metrics.get('energy_consumption', 0),
            'overhead': metrics.get('overhead_percentage', 0),
            'avg_waiting': metrics.get('avg_waiting', 0),
            'migrations': metrics.get('migration_count', 0)
        })

    results_df = pd.DataFrame(data)

    # تحلیل زمان پاسخ (اجرای سریع)
    analysis_data = []
    for cores in core_counts:
        tasks = generate_tasks(50, 0.5)  # نمونه کوچکتر
        avg_response = np.mean([t.period for t in tasks]) * 1.5  # جایگزین ساده
        analysis_data.append({
            'cores': cores,
            'simulation': avg_response,
            'closed_form': avg_response * 0.9,
            'milp': avg_response * 0.95
        })

    analysis_df = pd.DataFrame(analysis_data)

    return results_df, analysis_df

def plot_all_results(results_df, analysis_df, output_dir='results'):
    """ترسیم تمام نمودارهای مورد نیاز و ذخیره در مسیر مشخص"""
    # ایجاد دایرکتوری نتایج اگر وجود نداشته باشد
    os.makedirs(output_dir, exist_ok=True)

    # 1. نمودار تغییرات زمان پاسخ
    plt.figure(figsize=(14, 8))
    for policy in results_df['policy'].unique():
        subset = results_df[results_df['policy'] == policy]
        plt.plot(subset['num_tasks'], subset['avg_response'], 'o-', label=policy)

    plt.xlabel('Number of Tasks')
    plt.ylabel('Average Response Time (ms)')
    plt.title('Impact of Task Count on Response Time')
    plt.legend()
    plt.grid(True)
    plt.savefig(f'{output_dir}/response_time_vs_tasks.png', dpi=300)
    plt.close()

    # 2. نمودار نرخ رعایت مهلت
    plt.figure(figsize=(14, 8))
    cores = results_df['cores'].unique()
    width = 0.2
    x = np.arange(len(cores))

    for i, policy in enumerate(results_df['policy'].unique()):
        success_rates = []
        for core in cores:
            subset = results_df[(results_df['policy'] == policy) & (results_df['cores'] == core)]
            if not subset.empty:
                success_rates.append(subset['success_rate'].mean() * 100)
            else:
                success_rates.append(0)
        plt.bar(x + i*width, success_rates, width=width, label=policy)

    plt.xticks(x + width, cores)
    plt.xlabel('Number of Cores')
    plt.ylabel('Deadline Meeting Rate (%)')
    plt.title('Deadline Meeting Rate in Different Systems')
    plt.legend()
    plt.grid(True, axis='y')
    plt.savefig(f'{output_dir}/deadline_success_rate.png', dpi=300)
    plt.close()

    # 3. تأثیر بهره‌وری بر زمان پاسخ و انرژی
    fig, ax1 = plt.subplots(figsize=(14, 8))

    for policy in results_df['policy'].unique():
        subset = results_df[(results_df['policy'] == policy) & (results_df['cores'] == 8)]
        ax1.plot(subset['utilization'], subset['avg_response'], 'o-', label=f'{policy} (Response)')

    ax2 = ax1.twinx()
    for policy in results_df['policy'].unique():
        subset = results_df[(results_df['policy'] == policy) & (results_df['cores'] == 8)]
        ax2.plot(subset['utilization'], subset['energy'], 's--', label=f'{policy} (Energy)')

    ax1.set_xlabel('CPU Utilization')
    ax1.set_ylabel('Average Response Time (ms)')
    ax2.set_ylabel('Energy Consumption (J)')
    plt.title('Impact of Utilization on Response Time and Energy')
    fig.legend(loc='upper center', bbox_to_anchor=(0.5, 0.95), ncol=3)
    plt.grid(True)
    plt.savefig(f'{output_dir}/utilization_impact.png', dpi=300, bbox_inches='tight')
    plt.close()

    # 4. تحلیل تأخیرهای زمانبندی
    plt.figure(figsize=(14, 8))
    for policy in results_df['policy'].unique():
        subset = results_df[results_df['policy'] == policy]
        plt.plot(subset['num_tasks'], subset['avg_waiting'], 'D-', label=policy)

    plt.xlabel('Number of Tasks')
    plt.ylabel('Average Waiting Time (ms)')
    plt.title('Scheduling-Induced Delays')
    plt.legend()
    plt.grid(True)
    plt.savefig(f'{output_dir}/scheduling_delays.png', dpi=300)
    plt.close()

    # 5. سربار پردازشی
    plt.figure(figsize=(14, 8))
    for policy in results_df['policy'].unique():
        subset = results_df[results_df['policy'] == policy]
        plt.plot(subset['num_tasks'], subset['overhead'], 's--', label=policy)

    plt.xlabel('Number of Tasks')
    plt.ylabel('Scheduling Overhead (%)')
    plt.title('Processing Overhead of Different Methods')
    plt.legend()
    plt.grid(True)
    plt.savefig(f'{output_dir}/processing_overhead.png', dpi=300)
    plt.close()

    # 6. نتایج تحلیل زمان پاسخ
    plt.figure(figsize=(14, 8))
    plt.plot(analysis_df['cores'], analysis_df['closed_form'], 'o-', label='Closed-Form')
    plt.plot(analysis_df['cores'], analysis_df['milp'], 's--', label='MILP')
    plt.plot(analysis_df['cores'], analysis_df['simulation'], 'D-.', label='Simulation')

    plt.xlabel('Number of Cores')
    plt.ylabel('Average Response Time (ms)')
    plt.title('Response Time Analysis Methods Comparison')
    plt.legend()
    plt.grid(True)
    plt.savefig(f'{output_dir}/analysis_methods_comparison.png', dpi=300)
    plt.close()

# ==================== اجرای اصلی ====================
if __name__ == "__main__":
    # اتصال به Google Drive
    drive.mount('/content/drive')

    # تنظیم مسیر ذخیره‌سازی در Google Drive
    output_dir = "/content/drive/MyDrive/RT_Project/results"
    os.makedirs(output_dir, exist_ok=True)

    # اجرای آزمایش‌های جامع
    print("Starting comprehensive experiments...")
    results_df, analysis_df = run_comprehensive_experiments()

    # ذخیره نتایج
    results_df.to_csv(f'{output_dir}/comprehensive_results.csv', index=False)
    analysis_df.to_csv(f'{output_dir}/response_analysis.csv', index=False)

    # تولید نمودارها
    print("Generating plots...")
    plot_all_results(results_df, analysis_df, output_dir)

    print(f"Phase 2 completed successfully. Results saved in Google Drive: '{output_dir}'")

    # برای دانلود نتایج از Colab
    !zip -r /content/results.zip {output_dir}
    from google.colab import files
    files.download('/content/results.zip')

Mounted at /content/drive
Starting comprehensive experiments...
Starting parallel execution with 2 processes...


Running experiments:   0%|          | 0/48 [00:00<?, ?it/s]

Using device: cudaUsing device: cuda

Using device: cuda
Using device: cuda
Using device: cuda


Running experiments:   2%|▏         | 1/48 [00:00<00:08,  5.39it/s]

Using device: cuda
Using device: cuda


...

Running experiments: 100%|██████████| 48/48 [01:50<00:00,  2.30s/it]


Generating plots...


Phase 2 completed successfully. Results saved in Google Drive: '/content/drive/MyDrive/RT_Project/results'
  adding: content/drive/MyDrive/RT_Project/results/ (stored 0%)
  adding: content/drive/MyDrive/RT_Project/results/comprehensive_results.csv (deflated 56%)
  adding: content/drive/MyDrive/RT_Project/results/response_analysis.csv (deflated 29%)
  adding: content/drive/MyDrive/RT_Project/results/response_time_vs_tasks.png (deflated 17%)
  adding: content/drive/MyDrive/RT_Project/results/deadline_success_rate.png (deflated 44%)
  adding: content/drive/MyDrive/RT_Project/results/utilization_impact.png (deflated 12%)
  adding: content/drive/MyDrive/RT_Project/results/scheduling_delays.png (deflated 42%)
  adding: content/drive/MyDrive/RT_Project/results/processing_overhead.png (deflated 22%)
  adding: content/drive/MyDrive/RT_Project/results/analysis_methods_comparison.png (deflated 20%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>